# Smooth L1 loss

The current mlpack implementation is correct. <br>
This file just represents a minor modification to support None reduction.

### Imports and installation of mlpack

In [ ]:
%%capture
!sudo apt-get install libmlpack-dev 
import torch
import torch.nn as nn

### PyTorch

#### None Reduction


In [ ]:
loss = torch.nn.SmoothL1Loss(reduction='none')
input = torch.tensor([[-0.0494,  1.6028,  0.9639],
                      [-1.1958,  0.0737,  0.9648],
                      [-1.0486, -0.7091,  0.0745],
                      [-0.2121,  0.8612,  0.5924]], requires_grad=True)
target = torch.tensor([[ 0.4316,  0.5106,  0.7059],
                      [ 0.0164,  0.9255, -0.8288],
                      [-0.4478,  0.5571, -0.0231],
                      [ 1.1452,  0.0864, -1.0526]])
output = loss(input, target)
output.backward(torch.ones(input.shape))
print("Input : ")
print(input)
print("Target : ")
print(target)
print("FORWARD : ")
print("Loss : ")
print(output)
print("BACKWARD : ")
print(input.grad)

Input : 
tensor([[-0.0494,  1.6028,  0.9639],
        [-1.1958,  0.0737,  0.9648],
        [-1.0486, -0.7091,  0.0745],
        [-0.2121,  0.8612,  0.5924]], requires_grad=True)
Target : 
tensor([[ 0.4316,  0.5106,  0.7059],
        [ 0.0164,  0.9255, -0.8288],
        [-0.4478,  0.5571, -0.0231],
        [ 1.1452,  0.0864, -1.0526]])
FORWARD : 
Loss : 
tensor([[0.1157, 0.5922, 0.0333],
        [0.7122, 0.3628, 1.2936],
        [0.1805, 0.7662, 0.0048],
        [0.8573, 0.3002, 1.1450]], grad_fn=<SmoothL1LossBackward>)
BACKWARD : 
tensor([[-0.4810,  1.0000,  0.2580],
        [-1.0000, -0.8518,  1.0000],
        [-0.6008, -1.0000,  0.0976],
        [-1.0000,  0.7748,  1.0000]])


#### Sum Reduction

In [ ]:
loss = torch.nn.SmoothL1Loss(reduction='sum')
input = torch.tensor([[-0.0494,  1.6028,  0.9639],
                      [-1.1958,  0.0737,  0.9648],
                      [-1.0486, -0.7091,  0.0745],
                      [-0.2121,  0.8612,  0.5924]], requires_grad=True)
target = torch.tensor([[ 0.4316,  0.5106,  0.7059],
                      [ 0.0164,  0.9255, -0.8288],
                      [-0.4478,  0.5571, -0.0231],
                      [ 1.1452,  0.0864, -1.0526]])
output = loss(input, target)
output.backward()
print("Input : ")
print(input)
print("Target : ")
print(target)
print("FORWARD : ")
print("Loss : ")
print(output)
print("BACKWARD : ")
print(input.grad)

Input : 
tensor([[-0.0494,  1.6028,  0.9639],
        [-1.1958,  0.0737,  0.9648],
        [-1.0486, -0.7091,  0.0745],
        [-0.2121,  0.8612,  0.5924]], requires_grad=True)
Target : 
tensor([[ 0.4316,  0.5106,  0.7059],
        [ 0.0164,  0.9255, -0.8288],
        [-0.4478,  0.5571, -0.0231],
        [ 1.1452,  0.0864, -1.0526]])
FORWARD : 
Loss : 
tensor(6.3636, grad_fn=<SmoothL1LossBackward>)
BACKWARD : 
tensor([[-0.4810,  1.0000,  0.2580],
        [-1.0000, -0.8518,  1.0000],
        [-0.6008, -1.0000,  0.0976],
        [-1.0000,  0.7748,  1.0000]])


#### Mean reduction

In [ ]:
loss = torch.nn.SmoothL1Loss(reduction='mean')
input = torch.tensor([[-0.0494,  1.6028,  0.9639],
                      [-1.1958,  0.0737,  0.9648],
                      [-1.0486, -0.7091,  0.0745],
                      [-0.2121,  0.8612,  0.5924]], requires_grad=True)
target = torch.tensor([[ 0.4316,  0.5106,  0.7059],
                      [ 0.0164,  0.9255, -0.8288],
                      [-0.4478,  0.5571, -0.0231],
                      [ 1.1452,  0.0864, -1.0526]])
output = loss(input, target)
output.backward()
print("Input : ")
print(input)
print("Target : ")
print(target)
print("FORWARD : ")
print("Loss : ")
print(output)
print("BACKWARD : ")
print(input.grad)

Input : 
tensor([[-0.0494,  1.6028,  0.9639],
        [-1.1958,  0.0737,  0.9648],
        [-1.0486, -0.7091,  0.0745],
        [-0.2121,  0.8612,  0.5924]], requires_grad=True)
Target : 
tensor([[ 0.4316,  0.5106,  0.7059],
        [ 0.0164,  0.9255, -0.8288],
        [-0.4478,  0.5571, -0.0231],
        [ 1.1452,  0.0864, -1.0526]])
FORWARD : 
Loss : 
tensor(0.5303, grad_fn=<SmoothL1LossBackward>)
BACKWARD : 
tensor([[-0.0401,  0.0833,  0.0215],
        [-0.0833, -0.0710,  0.0833],
        [-0.0501, -0.0833,  0.0081],
        [-0.0833,  0.0646,  0.0833]])


### mlpack


#### CURRENT IMPLEMENTATION


In [ ]:
%%capture
%%writefile test.cpp  

#include <iostream>
#include <armadillo>

using namespace std;
using namespace arma;

int main()
{
  // Constructor
  arma::mat x,y;
  arma::mat weight;
  const double delta = 1.0;
 
  x << -0.0494 << 1.6028 << 0.9639 << endr
    << -1.1958 << 0.0737 << 0.9648 << endr
    << -1.0486 << -0.7091 << 0.0745 << endr
    << -0.2121 << 0.8612 << 0.5924 << endr;

  y << 0.4316  << 0.5106 <<  0.7059 << endr
    << 0.0164  << 0.9255 << -0.8288 << endr
    << -0.4478 << 0.5571 << -0.0231 << endr
    << 1.1452  << 0.0864 << -1.0526 << endr;

  // Forward
  double loss_sum = 0;
  for (size_t i = 0; i < x.n_elem; ++i)
  {
      const double absError = std::abs(y[i] - x[i]);
      loss_sum += absError > delta ? delta * (absError - 0.5 * delta) : 0.5 * std::pow(absError, 2);
  }
  double loss_mean = loss_sum / x.n_elem;

  // Backward
  arma::mat output;
  output.set_size(size(x));
  for (size_t i = 0; i < output.n_elem; ++i)
  {
    const double absErrorBack = std::abs(y[i] - x[i]);
    output[i] = absErrorBack > delta ? - delta * (y[i] - x[i]) / absErrorBack : x[i] - y[i];
  }

  // Display
  cout << "------------------------------------------------------------------" << endl;
  cout << "USER-PROVIDED MATRICES : " << endl;
  cout << "------------------------------------------------------------------" << endl;
  cout << "Input shape : "<< x.n_rows << " " << x.n_cols << endl;
  cout << "Input : " << endl << x << endl;
  cout << "Target shape : "<< y.n_rows << " " << y.n_cols << endl;
  cout << "Target : " << endl << y << endl;
  cout << "------------------------------------------------------------------" << endl;
  cout << "SUM " << endl;
  cout << "------------------------------------------------------------------" << endl;
  cout << "FORWARD : " << endl;
  cout << "Loss (sum):\n" << loss_sum << '\n';
  cout << "BACKWARD : " << endl;
  cout << "Output shape : "<< output.n_rows << " " << output.n_cols << endl;
  cout << "Output (sum) : " << endl << output << endl;
  cout << "Sum of all values in this matrix : " << arma::as_scalar(arma::accu(output)) << endl;
  cout << "------------------------------------------------------------------" << endl;
  cout << "MEAN " << endl;
  cout << "------------------------------------------------------------------" << endl;
  cout << "FORWARD : " << endl;
  cout << "Loss (mean):\n" << loss_mean << '\n';
  cout << "BACKWARD : " << endl;
  cout << "Output shape : "<< output.n_rows << " " << output.n_cols << endl;
  cout << "Output (mean) : " << endl << output / x.n_elem << endl;
  cout << "Sum of all values in this matrix : " << arma::as_scalar(arma::accu(output / x.n_elem)) << endl;
  cout << "------------------------------------------------------------------" << endl;
  return 0;
}

In [ ]:
%%script bash
g++ test.cpp -o test -larmadillo && ./test

------------------------------------------------------------------
USER-PROVIDED MATRICES : 
------------------------------------------------------------------
Input shape : 4 3
Input : 
  -0.0494   1.6028   0.9639
  -1.1958   0.0737   0.9648
  -1.0486  -0.7091   0.0745
  -0.2121   0.8612   0.5924

Target shape : 4 3
Target : 
   0.4316   0.5106   0.7059
   0.0164   0.9255  -0.8288
  -0.4478   0.5571  -0.0231
   1.1452   0.0864  -1.0526

------------------------------------------------------------------
SUM 
------------------------------------------------------------------
FORWARD : 
Loss (sum):
6.36364
BACKWARD : 
Output shape : 4 3
Output (sum) : 
  -0.4810   1.0000   0.2580
  -1.0000  -0.8518   1.0000
  -0.6008  -1.0000   0.0976
  -1.0000   0.7748   1.0000

Sum of all values in this matrix : -0.8032
------------------------------------------------------------------
MEAN 
------------------------------------------------------------------
FORWARD : 
Loss (mean):
0.530304
BACKWARD : 


#### NEW IMPLEMENTATION - Supports None Reduction

In [ ]:
%%capture
%%writefile test.cpp  

#include <iostream>
#include <armadillo>

using namespace std;
using namespace arma;

int main()
{
  // Constructor
  arma::mat x,y;
  arma::mat weight;
  const double delta = 1.0;
 
  x << -0.0494 << 1.6028 << 0.9639 << endr
    << -1.1958 << 0.0737 << 0.9648 << endr
    << -1.0486 << -0.7091 << 0.0745 << endr
    << -0.2121 << 0.8612 << 0.5924 << endr;

  y << 0.4316  << 0.5106 <<  0.7059 << endr
    << 0.0164  << 0.9255 << -0.8288 << endr
    << -0.4478 << 0.5571 << -0.0231 << endr
    << 1.1452  << 0.0864 << -1.0526 << endr;

  // Forward
  arma::mat loss_none;
  loss_none.set_size(size(x));
  for (size_t i = 0; i < x.n_elem; ++i)
  {
      const double absError = std::abs(y[i] - x[i]);
      loss_none[i] = absError > delta ? delta * (absError - 0.5 * delta) : 0.5 * std::pow(absError, 2);
  }
  double loss_sum = arma::accu(loss_none);
  double loss_mean = loss_sum / x.n_elem;

  // Backward
  arma::mat output;
  output.set_size(size(x));
  for (size_t i = 0; i < output.n_elem; ++i)
  {
    const double absErrorBack = std::abs(y[i] - x[i]);
    output[i] = absErrorBack > delta ? - delta * (y[i] - x[i]) / absErrorBack : x[i] - y[i];
  }

  // Display
  cout << "------------------------------------------------------------------" << endl;
  cout << "USER-PROVIDED MATRICES : " << endl;
  cout << "------------------------------------------------------------------" << endl;
  cout << "Input shape : "<< x.n_rows << " " << x.n_cols << endl;
  cout << "Input : " << endl << x << endl;
  cout << "Target shape : "<< y.n_rows << " " << y.n_cols << endl;
  cout << "Target : " << endl << y << endl;
  cout << "------------------------------------------------------------------" << endl;
  cout << "SUM " << endl;
  cout << "------------------------------------------------------------------" << endl;
  cout << "FORWARD : " << endl;
  cout << "Loss (none): \n" << loss_none << '\n';
  cout << "Loss (sum):\n" << loss_sum << '\n';
  cout << "BACKWARD : " << endl;
  cout << "Output shape : "<< output.n_rows << " " << output.n_cols << endl;
  cout << "Output (sum) : " << endl << output << endl;
  cout << "Sum of all values in this matrix : " << arma::as_scalar(arma::accu(output)) << endl;
  cout << "------------------------------------------------------------------" << endl;
  cout << "MEAN " << endl;
  cout << "------------------------------------------------------------------" << endl;
  cout << "FORWARD : " << endl;
  cout << "Loss (mean):\n" << loss_mean << '\n';
  cout << "BACKWARD : " << endl;
  cout << "Output shape : "<< output.n_rows << " " << output.n_cols << endl;
  cout << "Output (mean) : " << endl << output / x.n_elem << endl;
  cout << "Sum of all values in this matrix : " << arma::as_scalar(arma::accu(output / x.n_elem)) << endl;
  cout << "------------------------------------------------------------------" << endl;
  return 0;
}

In [ ]:
%%script bash
g++ test.cpp -o test -larmadillo && ./test

------------------------------------------------------------------
USER-PROVIDED MATRICES : 
------------------------------------------------------------------
Input shape : 4 3
Input : 
  -0.0494   1.6028   0.9639
  -1.1958   0.0737   0.9648
  -1.0486  -0.7091   0.0745
  -0.2121   0.8612   0.5924

Target shape : 4 3
Target : 
   0.4316   0.5106   0.7059
   0.0164   0.9255  -0.8288
  -0.4478   0.5571  -0.0231
   1.1452   0.0864  -1.0526

------------------------------------------------------------------
SUM 
------------------------------------------------------------------
FORWARD : 
Loss (none): 
   0.1157   0.5922   0.0333
   0.7122   0.3628   1.2936
   0.1805   0.7662   0.0048
   0.8573   0.3002   1.1450

Loss (sum):
6.36364
BACKWARD : 
Output shape : 4 3
Output (sum) : 
  -0.4810   1.0000   0.2580
  -1.0000  -0.8518   1.0000
  -0.6008  -1.0000   0.0976
  -1.0000   0.7748   1.0000

Sum of all values in this matrix : -0.8032
----------------------------------------------------------